In [1]:
from MyCreds.mycreds import IowaData
import pandas as pd
from sodapy import Socrata # < Unmaintained as of August 31, 2022, just in time for this project
import numpy as np
import h3
import requests
from geopy.geocoders import Nominatim

KEY = IowaData.api_key
SECRET = IowaData.api_key_secret
TOKEN = IowaData.app_token

liquor_sales_data_id = "m3tr-qhgy"
LIMIT = 24000000

In [8]:
# url = f"https://data.iowa.gov/resource/m3tr-qhgy.json?$$app_token={TOKEN}&$limit=20&$where=(vendor_name like '%10th Mountain%')"
url = f"https://data.iowa.gov/resource/m3tr-qhgy.json?$$app_token={TOKEN}&$query=select count(distinct(date))"
r = requests.get(url=url).json()
r

[{'count_distinct_date': '2706'}]

In [12]:

query = "select distinct(im_desc) limit 50000"
url = f"https://data.iowa.gov/resource/m3tr-qhgy.json?$$app_token={TOKEN}&$query={query}"
r = requests.get(url=url).json()
df = pd.DataFrame(r)
df

,im_desc_1
0,100 Anos Anejo
1,100 Anos Azul 100% Agave
2,10 Cane Rum
3,10th Mountain Bourbon
4,135° East Hyogo Japanese Dry Gin
...,...
10786,Zu Bison Grass Vodka
10787,Zubrowka Bison Grass Vodka
10788,Zwack
10789,Zwack Unicum


In [23]:
query = f"select date_extract_y(date), sum(sale_bottles), sum(sale_dollars), sum(sale_gallons) group by date_extract_y(date)"
url = f"https://data.iowa.gov/resource/m3tr-qhgy.json?$$app_token={TOKEN}&$query={query}"
r = requests.get(url=url).json()
df = pd.DataFrame(r)
df

,date_extract_y_date,sum_sale_bottles,sum_sale_dollars,sum_sale_gallons
0,2012,19917404,255203313.57,4954851.80
1,2013,19934692,258103792.82,4907690.85
2,2014,20666259,270587738.09,5061262.51
3,2015,21572443,284328572.11,5182900.41
4,2016,22670318,299447564.85,5363088.93
5,2017,23990017,314530044.81,5573698.52
6,2018,25435040,334216843.07,5779170.81
7,2019,26844339,349220341.63,5878523.32
8,2020,29841264,396663131.66,6381500.11
9,2021,31203537,428123535.40,6524942.92


In [24]:
df.plot()

TypeError: no numeric data to plot

In [25]:
df.dtypes

date_extract_y_date    object
sum_sale_bottles       object
sum_sale_dollars       object
sum_sale_gallons       object
dtype: object

___

___

In [21]:
from MyCreds.mycreds import IowaData
import pandas as pd
from sodapy import Socrata # < Unmaintained as of August 31, 2022, just in time for this project
import numpy as np
import h3
import requests
from geopy.geocoders import Nominatim
import pymssql

# Globals
geolocator = Nominatim(user_agent="my_user_agent")
KEY = IowaData.api_key
SECRET = IowaData.api_key_secret
TOKEN = IowaData.app_token
liquor_sales_data_id = "m3tr-qhgy"
LIMIT = 24000000

In [2]:
def get_unique_with_least_nulls(df):
    df['n_null'] = df.isnull().sum(axis=1)
    df = df.sort_values(['store_id', 'n_null'], ascending=False).groupby('store_id').last().reset_index()
    df.drop('n_null', axis=1, inplace=True)
    return df

In [3]:
def get_nominatim_lat_long(x):

    try:
        address = (x['address'] + ' ' + x['city'] + ', IA ' + x['zipcode']).replace('  ', ' ').strip()
        g = geolocator.geocode(address)
        if g:
            long, lat = g.longitude, g.latitude
        else:
            long, lat = np.nan, np.nan
    except:
        long, lat = np.nan, np.nan
        next


    return long, lat

In [4]:
def fill_missing_locs(df):
    missing = df[(~df['address'].isnull()) & (df['long'].isnull())]
    ll = missing.apply(lambda x: get_nominatim_lat_long(x), axis=1, result_type='expand')
    missing = missing.merge(ll, left_index=True, right_index=True).drop(['long', 'lat'], axis=1).rename({0: 'long', 1: 'lat'}, axis=1)

    df = df.merge(missing[['store_id', 'long', 'lat']], on='store_id', how='left')

    df['long_x'].fillna(df['long_y'], inplace=True)
    df['lat_x'].fillna(df['lat_y'], inplace=True)
    df.rename({'long_x': 'long', 'lat_x': 'lat'}, axis=1, inplace=True)
    df.drop(['long_y', 'lat_y'], axis=1, inplace=True)

    return df

In [5]:
def clean_locations(df, h3_resolution=8):
    coords_df = pd.DataFrame(df['geometry'].to_dict()).T
    coords_df['coordinates'] = np.where(coords_df['coordinates'].isnull(), pd.Series([[]]*len(coords_df)), coords_df['coordinates']) # fill na with empty list
    df = df.merge(pd.DataFrame(coords_df['coordinates'].to_list(), columns=['long', 'lat']), left_index=True, right_index=True)

    df['store_name'] = df['store_name'].str.replace('"', '').str.title().str.replace("'S", "'s")
    df['city'] = df['city'].str.title()
    df['address'] = df['address'].str.title()
    df['store_name'].replace("( #\d+ / .+| / .+| #\d+)", "", regex=True, inplace=True)
    df['zipcode'].replace("-", "", regex=True, inplace=True)

    print('Removing Duplicates...')
    df = get_unique_with_least_nulls(df)

    print('Filling missing lat/long...')
    df = fill_missing_locs(df)

    def lat_long_to_h3(row):
        return h3.geo_to_h3(lat=row.lat,lng=row.long,resolution = h3_resolution)

    df['hex_id'] = df.apply(lat_long_to_h3, axis=1)
    df.drop('geometry', axis=1, inplace=True)

    return df

In [17]:
def get_store_info():

    query = f"select distinct(store) as store_id, name as store_name, address, city, zipcode, county_number as county_id, store_location as geometry limit {LIMIT}"
    url = f"https://data.iowa.gov/resource/m3tr-qhgy.json?$$app_token={TOKEN}&$query={query}"
    print('Getting Request Response...')
    r = requests.get(url=url).json()
    df = pd.DataFrame(r)
    print('Cleaning Dataframe...')
    df = clean_locations(df)

    return df

In [18]:
store_map = get_store_info()
store_map

Getting Request Response...
Cleaning Dataframe...
Removing Duplicates...
Filling missing lat/long...


,store_id,store_name,address,city,zipcode,county_id,long,lat,hex_id
0,010004,Jack & Jill Store,115 East Main Street,West Branch,52358,None,-91.343394,41.671391,8826740da9fffff
1,010016,Kwik Star,511 1St Ave,Perry,50220,None,-94.106564,41.832534,88262b6d15fffff
2,010017,Kwik Star,1303 Main St,Emmetsburg,50536,None,-94.668298,43.111971,88262a1713fffff
3,010018,Kwik Star,1002 South County Rd,Toledo,52342,None,-92.581683,41.986706,882676d6dbfffff
4,010019,Oelwein Mart,801 East Charles Street,Oelwein,50662,None,-91.902413,42.678786,882629b80bfffff
...,...,...,...,...,...,...,...,...,...
2791,9934,"Cats Eye Distillery, Llc",4860 Heatherstone Rd.,Bettendorf,52722,82,-90.455001,41.571059,882675dadbfffff
2792,9936,Dash Events Llc,1685 W Uintah St. #101,Colorado Springs,80904,None,-104.845334,38.848017,88268c6f49fffff
2793,9937,"Green Frog Distillery, Llc",3345 270Th Ave.,Keokuk,52632,56,-91.399797,40.473242,8826702ac1fffff
2794,9938,Templeton Distilling Llc,209 E 3Rd St,Templeton,51463,14,-94.936366,41.917072,88260c040bfffff


In [19]:
store_map.columns

Index(['store_id', 'store_name', 'address', 'city', 'zipcode', 'county_id',
       'long', 'lat', 'hex_id'],
      dtype='object')

In [20]:
store_map['county_id'].value_counts()

77    362
57    191
82    121
52    104
07     89
     ... 
05      3
5       2
1       2
02      1
2       1
Name: county_id, Length: 108, dtype: int64

In [22]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def load(df):
    server = IowaData.azure_server
    user = IowaData.azure_uid
    password = IowaData.azure_pwd

    write_data = list(map(tuple, df.fillna(0).values))
    insert_query = f"""INSERT INTO invoices.store({', '.join([*df.columns])}) VALUES ({', '.join(['%s' for col in df.columns])})"""
    insert_q, values_q = insert_query.split('VALUES') # get part with the query and the parameters
    insert_q += 'VALUES' # add values to make sql query correct after split

    with pymssql.connect(server, user, password, "iowa_liquor") as cnxn:
        with cnxn.cursor() as cur:
            for chunk_data in chunks(write_data, 1000):
                # chunk_data contains list of row parameters
                # we make it flat to use execute later instead execute_many
                flat_list = [item for sublist in chunk_data for item in sublist]
                # creating the query with multiple values insert
                chunk_query = insert_q + ','.join([values_q] * len(chunk_data))
                cur.execute(chunk_query, tuple(flat_list))
                cnxn.commit()


In [24]:
store_map['zipcode'].replace("-", "", regex=True, inplace=True)

In [26]:
store_map[store_map['zipcode'] == '7122']

,store_id,store_name,address,city,zipcode,county_id,long,lat,hex_id
766,4307,Crossroads Wine And Liquor,117 Iowa Ave,Dunlap,7122,43,-95.602969,41.854728,88260c4ebbfffff
